In [4]:
!pip install -q openpyxl tqdm

In [5]:
import pandas as pd

merged_data = pd.read_excel("/content/merged_data_english_only.xlsx")

In [6]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

# 📦 Chargement et prétraitement
def clean_text(text):
    import re
    text = re.sub(r"<br\s*/?>", " ", str(text))
    text = re.sub(r"[^\w\s]", "", text)
    return text.lower().strip()

# 🧠 Charger les données
df = merged_data[['description_translated', 'comments_translated']].dropna()

# 🔁 Regrouper les commentaires par description
df_grouped = df.groupby("description_translated")["comments_translated"].apply(lambda x: ' '.join(x)).reset_index()
df_grouped.columns = ["description", "all_comments"]

# 🔗 Fusionner les commentaires + description d’origine comme contexte
df_grouped["input_text"] = df_grouped["description"].apply(clean_text) + " " + df_grouped["all_comments"].apply(clean_text)

# 🔧 Charger le pipeline de résumé avec T5 (petit modèle pour commencer)
summarizer = pipeline("summarization", model="t5-small", tokenizer="t5-small")

# 🔁 Génération des descriptions avec barre de progression
tqdm.pandas(desc="✍️ Génération avec T5")
def t5_summarize(text):
    prompt = "summarize: " + text
    try:
        summary = summarizer(prompt, max_length=50, min_length=10, do_sample=False)[0]["summary_text"]
        return summary
    except Exception as e:
        return "Résumé indisponible"

df_grouped["generated_description"] = df_grouped["input_text"].progress_apply(t5_summarize)

# 💾 Sauvegarde
df_grouped.to_excel("descriptions_t5.xlsx", index=False)
print("✅ Descriptions enrichies avec T5 sauvegardées dans 'descriptions_t5.xlsx'")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0
✍️ Génération avec T5: 100%|██████████| 6017/6017 [1:04:57<00:00,  1.54it/s]


✅ Descriptions enrichies avec T5 sauvegardées dans 'descriptions_t5.xlsx'
